In [1]:
import csv

# baca data csv , simpan ke bentuk list
with open("Fitur/area.csv") as f:
    reader = csv.reader(f)
    # fungsi list() untuk mengubah ke bentuk list()
    area = list(reader)[0]
    
with open("Fitur/eccentricity.csv") as f:
    reader = csv.reader(f)
    eccentricity = list(reader)[0]

with open("Fitur/perimeter.csv") as f:
    reader = csv.reader(f)
    perimeter = list(reader)[0]
    
with open("Fitur/metric.csv") as f:
    reader = csv.reader(f)
    metric = list(reader)[0]
    
with open("Fitur/red.csv") as f:
    reader = csv.reader(f)
    red = list(reader)[0]
    
with open("Fitur/green.csv") as f:
    reader = csv.reader(f)
    green = list(reader)[0]
    
with open("Fitur/blue.csv") as f:
    reader = csv.reader(f)
    blue = list(reader)[0]

with open("Fitur/blue.csv") as f:
    reader = csv.reader(f)
    blue = list(reader)[0]
    
with open("Fitur/label.csv") as f:
    reader = csv.reader(f)
    label = list(reader)[0]
    
# ganti label dalam bentuk string
idx = 0
for i in label:
    if i == "1":
        label[idx] = "Cocor Bebek"
    elif i == "2":
        label[idx] = "Jambu"
    elif i == "3":
        label[idx] = "Kacang Panjang"
    elif i == "4":
        label[idx] = "Singkong"
    elif i == "5":
        label[idx] = "Tomat"
        
    idx = idx+1

In [2]:
import pandas as pd
# fungsi map() untuk mengubah tipe data semua element dalam list
# fungsi zip() untuk menggandengkan setiap list menjadi matriks
df = pd.DataFrame(list(zip(list(map(int,area)), list(map(float,eccentricity)), list(map(float,perimeter)), list(map(float,metric)), list(map(float,red)), list(map(float,green)), list(map(float,blue)), label)), columns = ['Area', 'Eccentricity', 'Perimeter', 'Metric', 'Red', 'Green', 'Blue', 'Label'])

In [3]:
# melihat 5 data terakhir, sekedar melihat struktur tabel
df.tail()

,Area,Eccentricity,Perimeter,Metric,Red,Green,Blue,Label
95,2592,0.587480,648.558,0.077437,62.147971,98.496181,34.956567,Tomat
96,2034,0.744930,296.505,0.290735,48.569492,67.977778,38.859887,Tomat
97,1796,0.736844,262.322,0.327980,82.097821,107.617685,66.557027,Tomat
98,1931,0.789033,542.651,0.082404,62.952130,88.649493,54.805680,Tomat
99,1787,0.792165,525.614,0.081283,80.768362,111.433290,70.402434,Tomat


In [4]:
# memisahkan data ke train dan test
train_dataset = df.sample(frac=0.8, random_state=0)
test_dataset = df.drop(train_dataset.index)

In [5]:
# memisahkan label
train_features = train_dataset.copy()
test_features = test_dataset.copy()
# pd.get_dummies() untuk membuat one-hot encoding
train_labels = pd.get_dummies(train_features.pop('Label'))
test_labels = pd.get_dummies(test_features.pop('Label'))

In [6]:
# membuat neural network
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import preprocessing
import numpy as np

# membuat normalization layer
normalizer = preprocessing.Normalization()

normalizer.adapt(np.array(train_features))

In [7]:
first = np.array(train_features[:1])
# mengecek apak normalization layer sudah berfungsi
with np.printoptions(precision=2, suppress=True):
    print('First example:',first)
    print()
    print('Normalized:', normalizer(first).numpy())

First example: [[3843.      0.78  846.89    0.07   41.99   85.28   22.97]]

Normalized: [[ 0.88 -0.2   2.27 -1.38 -1.12  0.66 -0.62]]


In [9]:
# kode ini mengikuti tutorial dari machinelearningmastery.com 
# https://machinelearningmastery.com/multi-class-classification-tutorial-keras-deep-learning-library/
# saya coba untuk run beberapa kali, nilai accuracy dapat berubah-ubah
def baseline_model():
    model = keras.models.Sequential()
    model.add(normalizer)
    model.add(layers.Dense(14, input_dim=7, activation='relu'))
    model.add(layers.Dense(5, activation='softmax'))
    # compile model
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

estimator = KerasClassifier(build_fn=baseline_model, epochs=80, batch_size=5, verbose=0)
kfold = KFold(n_splits=10, shuffle=True)
results = cross_val_score(estimator, train_features, train_labels, cv=kfold)
print("Accuracy: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Accuracy: 81.25% (15.05%)
